# Libraries and Data

In [29]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler

In [30]:
# Load the data
# YYYY-MM-DD
df = pd.read_csv('../nyc_data.csv', index_col = 0, parse_dates = True)
df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [31]:
# Rename variable
df = df.rename({'Demand': 'y'}, axis = 1)
df

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077
...,...,...,...,...,...,...
2020-12-27,685.915026,0,0,0,2.89,38.674
2020-12-28,998.051170,0,0,0,8.83,166.712
2020-12-29,847.123399,0,0,0,3.48,161.865


# Prepare for LSTM

In [32]:
# Time Series object
series = TimeSeries.from_series(df.y) # Datapoint we are trying to predict
covariates = TimeSeries.from_dataframe(df.iloc[:, 1:])

In [33]:
# Year
year_series = datetime_attribute_timeseries(
    pd.date_range(start = series.start_time(),
                  freq = series.freq_str,
                  periods = df.shape[0]),
    attribute = "year",
    one_hot = False)

# Month
month_series = datetime_attribute_timeseries(year_series,
                                             attribute = "month",
                                             one_hot = True)

# Weekday
weekday_series = datetime_attribute_timeseries(year_series,
                                               attribute = "weekday",
                                               one_hot = True)

In [34]:
# Preparing scalers
transformer1 = Scaler()
transformer2 = Scaler()

In [35]:
# Scale the Y (timeseries)
y_transformed = transformer1.fit_transform(series)

In [36]:
# Sale the covariates
covariates = covariates.stack(year_series)
covariates_transformed =transformer2.fit_transform(covariates)
covariates_transformed = covariates_transfformed.stack(month_series)
covariates_transformed = covariates_transfformed.stack(weekday_series)

In [37]:
covariates_transformed

<TimeSeries (DataArray) (Date: 2192, component: 32, sample: 1)>
array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
...
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])
Coordinates:
  * Date       (Date) datetime64[ns] 2015-01-01 2015-01-02 ... 2020-12-31
  * component  (component) <U12 'Easter' 'Thanksgiving' ... 'weekday_7_1'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None